<a href="https://colab.research.google.com/github/Saurav2910/My-Deeplearning-Projects/blob/main/Urban_Sound_Classification_Spectrogram_Images_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Urban Sound Classification from Spectrogram Images with ConvNet

### UrbanSound8K

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: **air_conditioner, car_horn, children_playing, dog_bark, drilling, enginge_idling, gun_shot, jackhammer, siren, and street_music.**

Spectrogram features have been extracte from the audio and saved as images.

Subsequently, 2D Convolutions have been applied on these spectrogram images to classify the sounds into different class categories.


In [1]:
%tensorflow_version 2.x

In [2]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp36-none-any.whl size=30181 sha256=f09d62414fd5b0351ee99f09f6f13729cef262ec08fe4fd5b4c10ade27dea51f
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [3]:
%matplotlib inline
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np

In [4]:
# %%capture
!apt-get install libav-tools -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package libav-tools is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  ffmpeg

E: Package 'libav-tools' has no installation candidate


### Download the Zip file containing the Raw Audio Files (Wav Format)

In [5]:
!wget https://www.dropbox.com/s/qoyihkmmi55yd2j/fold1.zip

--2020-12-01 13:35:13--  https://www.dropbox.com/s/qoyihkmmi55yd2j/fold1.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/qoyihkmmi55yd2j/fold1.zip [following]
--2020-12-01 13:35:14--  https://www.dropbox.com/s/raw/qoyihkmmi55yd2j/fold1.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc50eba100c5735c86981b778005.dl.dropboxusercontent.com/cd/0/inline/BEMuuVfTh7k6uXEZu7iAASemTAATOR1bowoy-QyejIDoSQ4TM3lCHfeV-Njb5vqiCrtFzpKgh3k2zj-EcRNkcQXBRY3rXExGVT2hCzWk8O-SLg5GPOqUfmzJCYkiXydXYSA/file# [following]
--2020-12-01 13:35:14--  https://uc50eba100c5735c86981b778005.dl.dropboxusercontent.com/cd/0/inline/BEMuuVfTh7k6uXEZu7iAASemTAATOR1bowoy-QyejIDoSQ4TM3lCHfeV-Njb5vqiCrtFzpKgh3k2zj-EcRNkcQXBRY3rXExGVT2hCzWk8O-SLg5GPOqUfmzJCYk

### Extract the Audio

In [6]:
!unzip fold1.zip

Archive:  fold1.zip
   creating: fold1/
  inflating: __MACOSX/._fold1        
  inflating: fold1/135776-2-0-49.wav  
  inflating: __MACOSX/fold1/._135776-2-0-49.wav  
  inflating: fold1/46654-6-0-0.wav   
  inflating: __MACOSX/fold1/._46654-6-0-0.wav  
  inflating: fold1/57320-0-0-24.wav  
  inflating: __MACOSX/fold1/._57320-0-0-24.wav  
  inflating: fold1/134717-0-0-26.wav  
  inflating: __MACOSX/fold1/._134717-0-0-26.wav  
  inflating: fold1/174276-7-5-0.wav  
  inflating: __MACOSX/fold1/._174276-7-5-0.wav  
  inflating: fold1/180937-7-2-6.wav  
  inflating: __MACOSX/fold1/._180937-7-2-6.wav  
  inflating: fold1/17913-4-0-1.wav   
  inflating: __MACOSX/fold1/._17913-4-0-1.wav  
  inflating: fold1/103074-7-4-6.wav  
  inflating: __MACOSX/fold1/._103074-7-4-6.wav  
  inflating: fold1/176787-5-0-9.wav  
  inflating: __MACOSX/fold1/._176787-5-0-9.wav  
  inflating: fold1/55020-4-0-11.wav  
  inflating: __MACOSX/fold1/._55020-4-0-11.wav  
  inflating: fold1/138015-3-0-7.wav  
  inflating:

### Clean Up

In [7]:
!rm -rf /content/__MACOSX

In [8]:
!rm fold1.zip

In [9]:
!pip install path

In [10]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path


In [11]:
import os
os.mkdir('train')
os.mkdir('test')

## Custom Function for Extracting Feature (Spectrogram) from Audio and Save as Image Files

In [12]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    # Compute a mel-scaled spectrogram.
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = './train/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [13]:
glob('./train/*')

[]

In [14]:
file_list=list(glob("./fold1/*"))

In [15]:
file_list

['./fold1/57320-0-0-45.wav',
 './fold1/146186-5-0-7.wav',
 './fold1/146186-5-0-1.wav',
 './fold1/46669-4-0-24.wav',
 './fold1/182800-2-3-0.wav',
 './fold1/21684-9-0-39.wav',
 './fold1/103258-5-0-17.wav',
 './fold1/180937-7-2-3.wav',
 './fold1/159738-8-0-1.wav',
 './fold1/108041-9-0-2.wav',
 './fold1/197318-6-1-0.wav',
 './fold1/103074-7-4-6.wav',
 './fold1/134717-0-0-18.wav',
 './fold1/57320-0-0-7.wav',
 './fold1/176787-5-0-12.wav',
 './fold1/22962-4-0-0.wav',
 './fold1/194732-9-0-163.wav',
 './fold1/180937-7-1-11.wav',
 './fold1/176787-5-0-2.wav',
 './fold1/24074-1-0-4.wav',
 './fold1/113205-5-1-3.wav',
 './fold1/57320-0-0-22.wav',
 './fold1/57320-0-0-21.wav',
 './fold1/162703-1-0-0.wav',
 './fold1/89212-3-0-0.wav',
 './fold1/157867-8-0-17.wav',
 './fold1/24074-1-0-5.wav',
 './fold1/72261-3-0-17.wav',
 './fold1/46669-4-0-31.wav',
 './fold1/147764-4-4-1.wav',
 './fold1/108041-9-0-11.wav',
 './fold1/180937-7-4-3.wav',
 './fold1/157867-8-0-25.wav',
 './fold1/59277-0-0-0.wav',
 './fold1/3

### Create Spectrograms and Save as JPEG File

In [16]:
for i,file in enumerate(file_list):
    print(i)
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
glob('./train/*')

['./train/138031-2-0-47.jpg',
 './train/180937-7-2-9.jpg',
 './train/118279-8-0-3.jpg',
 './train/180256-3-0-3.jpg',
 './train/72261-3-0-6.jpg',
 './train/132016-9-0-16.jpg',
 './train/180256-3-0-1.jpg',
 './train/134717-0-0-2.jpg',
 './train/9031-3-2-0.jpg',
 './train/78360-4-0-20.jpg',
 './train/146186-5-0-0.jpg',
 './train/180937-7-3-31.jpg',
 './train/77766-9-0-0.jpg',
 './train/127873-0-0-0.jpg',
 './train/103074-7-4-6.jpg',
 './train/65749-3-1-12.jpg',
 './train/159738-8-0-15.jpg',
 './train/24074-1-0-2.jpg',
 './train/103258-5-0-15.jpg',
 './train/160009-2-0-22.jpg',
 './train/174290-6-2-0.jpg',
 './train/172314-9-0-22.jpg',
 './train/88466-7-0-0.jpg',
 './train/176787-5-0-6.jpg',
 './train/177621-0-0-80.jpg',
 './train/15564-2-0-1.jpg',
 './train/46669-4-0-3.jpg',
 './train/103074-7-4-1.jpg',
 './train/177621-0-0-128.jpg',
 './train/180937-7-3-14.jpg',
 './train/174290-6-3-0.jpg',
 './train/99180-9-0-36.jpg',
 './train/157867-8-0-10.jpg',
 './train/165645-4-5-0.jpg',
 './train/

### Download and Read the CSV File containing the MetaData

In [19]:
!wget https://www.dropbox.com/s/6bhm4sfumk5p8w4/UrbanSound8K.csv

--2020-12-01 13:59:22--  https://www.dropbox.com/s/6bhm4sfumk5p8w4/UrbanSound8K.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6bhm4sfumk5p8w4/UrbanSound8K.csv [following]
--2020-12-01 13:59:22--  https://www.dropbox.com/s/raw/6bhm4sfumk5p8w4/UrbanSound8K.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc09d0b6cdfec9edcbcacc580b21.dl.dropboxusercontent.com/cd/0/inline/BEMGwXyT2z5_ibu-ptGhNRNPqtXyDjMmiCyDCl6PddYc3Hr2Wajgn1pMAqHvpSo8_MVDq9glkgdgcoR0AFMOOyvLBcAIn_MkmmVZ-JDpZXOQEw/file# [following]
--2020-12-01 13:59:22--  https://uc09d0b6cdfec9edcbcacc580b21.dl.dropboxusercontent.com/cd/0/inline/BEMGwXyT2z5_ibu-ptGhNRNPqtXyDjMmiCyDCl6PddYc3Hr2Wajgn1pMAqHvpSo8_MVDq9glkgdgcoR0AFMOOyvLBcAIn_MkmmVZ-JDpZXOQEw/file
Resolvi

In [20]:
df=pd.read_csv('/content/UrbanSound8K.csv')

In [22]:
df.head(20)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


### Get the File Names from the DataFrame that are the part of the Training Dataset

In [23]:
file_names=[x.split('/')[-1] for x in file_list]

In [24]:
file_names

['57320-0-0-45.wav',
 '146186-5-0-7.wav',
 '146186-5-0-1.wav',
 '46669-4-0-24.wav',
 '182800-2-3-0.wav',
 '21684-9-0-39.wav',
 '103258-5-0-17.wav',
 '180937-7-2-3.wav',
 '159738-8-0-1.wav',
 '108041-9-0-2.wav',
 '197318-6-1-0.wav',
 '103074-7-4-6.wav',
 '134717-0-0-18.wav',
 '57320-0-0-7.wav',
 '176787-5-0-12.wav',
 '22962-4-0-0.wav',
 '194732-9-0-163.wav',
 '180937-7-1-11.wav',
 '176787-5-0-2.wav',
 '24074-1-0-4.wav',
 '113205-5-1-3.wav',
 '57320-0-0-22.wav',
 '57320-0-0-21.wav',
 '162703-1-0-0.wav',
 '89212-3-0-0.wav',
 '157867-8-0-17.wav',
 '24074-1-0-5.wav',
 '72261-3-0-17.wav',
 '46669-4-0-31.wav',
 '147764-4-4-1.wav',
 '108041-9-0-11.wav',
 '180937-7-4-3.wav',
 '157867-8-0-25.wav',
 '59277-0-0-0.wav',
 '31323-3-0-2.wav',
 '157867-8-0-26.wav',
 '174276-7-5-0.wav',
 '106905-8-0-3.wav',
 '118279-8-0-11.wav',
 '157867-8-0-0.wav',
 '180937-7-3-24.wav',
 '192124-2-0-0.wav',
 '157867-8-0-13.wav',
 '138015-3-0-7.wav',
 '101415-3-0-8.wav',
 '118279-8-0-12.wav',
 '9031-3-1-0.wav',
 '191431

### Take only the Records that is a part of the Training Set 

In [25]:
df=df[df['slice_file_name'].isin(file_names)]

In [26]:
df

,slice_file_name,fsID,start,end,salience,fold,classID,class
64,101415-3-0-2.wav,101415,1.000000,5.000000,1,1,3,dog_bark
65,101415-3-0-3.wav,101415,1.500000,5.500000,1,1,3,dog_bark
66,101415-3-0-8.wav,101415,4.000000,8.000000,1,1,3,dog_bark
105,102106-3-0-0.wav,102106,2.243852,3.884477,2,1,3,dog_bark
106,102305-6-0-0.wav,102305,0.000000,2.611610,1,1,6,gun_shot
...,...,...,...,...,...,...,...,...
8676,99180-9-0-2.wav,99180,1.000000,5.000000,1,1,9,street_music
8677,99180-9-0-36.wav,99180,18.000000,22.000000,1,1,9,street_music
8678,99180-9-0-48.wav,99180,24.000000,28.000000,1,1,9,street_music
8679,99180-9-0-49.wav,99180,24.500000,28.500000,1,1,9,street_music


### Check the Distinct Labels Categories and the Number of Instances

In [27]:
df['class'].value_counts()

jackhammer          120
air_conditioner     100
children_playing    100
dog_bark            100
drilling            100
street_music        100
engine_idling        96
siren                86
car_horn             36
gun_shot             35
Name: class, dtype: int64

In [28]:
classes=list(df['class'].value_counts().index)

In [29]:
classes

['jackhammer',
 'air_conditioner',
 'children_playing',
 'dog_bark',
 'drilling',
 'street_music',
 'engine_idling',
 'siren',
 'car_horn',
 'gun_shot']

### Create Subfolders inside the 'train' folder for each of the Output Category Labels

In [30]:
for cl in classes:
  os.mkdir('/content/train/'+cl)

In [31]:
class_dict_map={x:'/content/train/'+x+'/' for x in classes}

In [32]:
# Class labels and path from where to read the files
class_dict_map

{'air_conditioner': '/content/train/air_conditioner/',
 'car_horn': '/content/train/car_horn/',
 'children_playing': '/content/train/children_playing/',
 'dog_bark': '/content/train/dog_bark/',
 'drilling': '/content/train/drilling/',
 'engine_idling': '/content/train/engine_idling/',
 'gun_shot': '/content/train/gun_shot/',
 'jackhammer': '/content/train/jackhammer/',
 'siren': '/content/train/siren/',
 'street_music': '/content/train/street_music/'}

In [33]:
# The shutil module offers a number of high-level operations on files and collections of files. 
# In particular, functions are provided which support file copying and removal.
import shutil

In [35]:
def return_class(file_name):
  cl=df['class'][df['slice_file_name']==file_name]
  return(cl.values[0])

In [36]:
for file in file_names:
  name=file.split('.')[0]+'.jpg'
  shutil.move('/content/train/'+name,
              class_dict_map[return_class(file)]+name)

## Data Augmentation - Synthetic Data Generation

In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [38]:
WIDTH = 64
HEIGHT = 64
BATCH_SIZE = 32
TRAIN_DIR=r'/content/train'

# data prep
train_datagen = ImageDataGenerator( rescale=1./255.,validation_split=0.25)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(HEIGHT, WIDTH), batch_size=BATCH_SIZE, class_mode='categorical')


Found 873 images belonging to 10 classes.


## Build the ConvNet Model

In [39]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras import regularizers, optimizers


model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.RMSprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        3

In [40]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, epochs=100)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
27/27 [==============================] - 1s 42ms/step - loss: 2.2082 - accuracy: 0.1677
Epoch 2/100
27/27 [==============================] - 1s 39ms/step - loss: 1.9621 - accuracy: 0.2836
Epoch 3/100
27/27 [==============================] - 1s 39ms/step - loss: 1.8189 - accuracy: 0.3460
Epoch 4/100
27/27 [==============================] - 1s 41ms/step - loss: 1.6879 - accuracy: 0.3912
Epoch 5/100
27/27 [==============================] - 1s 40ms/step - loss: 1.5581 - accuracy: 0.4685
Epoch 6/100
27/27 [==============================] - 1s 40ms/step - loss: 1.4305 - accuracy: 0.5172
Epoch 7/100
27/27 [==============================] - 1s 41ms/step - loss: 1.2053 - accuracy: 0.5719
Epoch 8/100
27/27 [==============================] - 1s 41ms/step - loss: 1.0967 - accuracy: 0.6314
Epoch 9/100
27/27 [==============================] - 1s 41ms/step - loss: 1.0021 - accuracy: 0.6421
Epoch 10/100
27/27 [====